In [1]:
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unidecode
import re
from spacy.cli import download
import spacy
import nltk
import contractions
from sklearn.model_selection import train_test_split
import gensim

spacy.cli.download("en_core_web_sm")
nltk.download('stopwords')

from nltk.corpus import stopwords


2026-01-22 17:10:53.290853: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-22 17:10:53.722323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-22 17:10:55.364488: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/tguyot/PersonalCode/spam-classifier/.venv/lib/python3.12/site-packages/keras/src/expo

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.2 MB/s  0:00:00eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /home/tguyot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Import cleaned preprocess csv
df = pd.read_csv('dataset/cleaned/cleaned_spam.csv').drop(columns=['Unnamed: 0'])
df

,is_spam,sms
0,0,go point crazy available boris n great world l...
1,0,ok war joke
2,1,free entry wily come win cup final st may text...
3,0,dun say early c already say
4,0,ah I think go live around though
...,...,...
5548,1,nd time try contact win pound prize claim easy...
5549,0,I b go esplanade fr home
5550,0,pity mood suggestion
5551,0,guy itching I act like I interested buy someth...


In [4]:
def get_corpus(df):
    return ' '.join(' '.join(df.sms.tolist()).split())

def get_unique_words(df):
    corpus = get_corpus(df)
    unique = set()
    output = []
    for word in corpus.split():
        if word not in unique:
            unique.add(word)
            output.append(word)
    return output

def count_unique(df):
    return len(get_unique_words(df))



print(count_unique(df))

get_unique_words(df)[:10] 

5277


['go',
 'point',
 'crazy',
 'available',
 'boris',
 'n',
 'great',
 'world',
 'la',
 'e']

In [5]:
# WOrd2Vec Model
class NaiveWord2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.weight1 = tf.keras.layers.Dense(embedding_dim, input_shape=(vocab_size,), activation='linear', name='embedding_layer')
        self.weight2 = tf.keras.layers.Dense(vocab_size, input_shape=(embedding_dim,), activation='softmax')
    
    def call(self, inputs):
        x = self.weight1(inputs)
        x = self.weight2(x)
        return x

In [6]:
# Test with gensim first


X_train, X_test, y_train, y_test = train_test_split(df['sms'].apply(lambda x: x.split()),
                                                    df['is_spam'],
                                                    test_size=0.2,
                                                    random_state=42)
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=2,
                                   min_count=1)

# Test word vectors -> lots of 0.999 cosine similarities
w2v_model.wv.most_similar('mobile', topn=20)


[('txt', 0.9992980360984802),
 ('free', 0.9992956519126892),
 ('give', 0.9992565512657166),
 ('call', 0.999247133731842),
 ('make', 0.9992321729660034),
 ('phone', 0.9992314577102661),
 ('go', 0.9992285370826721),
 ('win', 0.9992283582687378),
 ('reply', 0.9992155432701111),
 ('number', 0.9991986751556396),
 ('get', 0.9991981983184814),
 ('week', 0.9991962313652039),
 ('day', 0.9991894364356995),
 ('god', 0.9991675019264221),
 ('n', 0.9991648197174072),
 ('place', 0.9991633892059326),
 ('b', 0.9991531372070312),
 ('text', 0.999146580696106),
 ('late', 0.999137282371521),
 ('take', 0.9991199970245361)]

In [7]:
# Continuous bag of words

def generate_cbows(text, window_size):
    """Generate Continuous Bag of Words (CBOW) pairs from the given text. 
    text: preprocessed sequence of text"""
    # Tokenize the text
    words = text.split()

    # Create CBOW pairs with a given window size
    cbows = []
    for i, target_word in enumerate(words):
        context_words = words[max(0, i - window_size):i] + words[i + 1:i + window_size + 1]
        if len(context_words) == window_size * 2:
            cbows.append((context_words, target_word))
    return cbows


In [8]:
def one_hot_encoding(word: str, unique_words: list):
    try:
        assert word in set(unique_words)
    except AssertionError:
        print(f"Word '{word}' not found in unique_words.")
        return None
    vector = np.zeros(len(unique_words))
    index = unique_words.index(word)
    vector[index] = 1
    return tf.convert_to_tensor(vector)


In [9]:
def prepare_cbow(df, window_size):
    # Create Cbows non-encoded
    cbows = generate_cbows(get_corpus(df), window_size=window_size)

    # One-hot-encode words
    unique_words = list(get_unique_words(df))
    one_hot_encodings = {
        word: one_hot_encoding(word, unique_words) for word in unique_words
    }

    # Convert CBOW pairs to vector pairs
    cbow_vector_pairs = [([one_hot_encodings[word] for word in context_words], one_hot_encodings[target_word]) for context_words, target_word in cbows]

    # Sum the context vectors to get a single context vector
    cbow_vector_pairs = [(tf.reduce_sum(tf.stack(context_vectors), axis=0), target_vector) for context_vectors, target_vector in cbow_vector_pairs]

    return cbow_vector_pairs


In [10]:
unique_words = list(get_unique_words(df))
VOCAB_SIZE = len(unique_words)
VECTOR_DIM = 250
WINDOW_SIZE = 2

model = NaiveWord2Vec(VOCAB_SIZE, VECTOR_DIM)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Get cbow pairs
cbow_vector_pairs = prepare_cbow(df, window_size=WINDOW_SIZE)

model.fit(
    x=tf.stack([pair[0] for pair in cbow_vector_pairs]),
    y=tf.stack([pair[1] for pair in cbow_vector_pairs]),
    epochs=15,
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)


/home/tguyot/PersonalCode/spam-classifier/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1769098335.093512  224202 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/15
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.0713 - loss: 7.1225 - val_accuracy: 0.0822 - val_loss: 6.8492
Epoch 2/15
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.1044 - loss: 6.2098 - val_accuracy: 0.1151 - val_loss: 6.5334
Epoch 3/15
 725/1180 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.1566 - loss: 5.4033

KeyboardInterrupt: 

In [ ]:
# Get word embeddings
embeddings = model.get_layer('embedding_layer').get_weights()[0]
word_embeddings = {word: embeddings[idx] for idx, word in enumerate(unique_words)
}

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Get ntop similar words
def get_top_n_similar_words(target_word, word_embeddings, n=10):
    target_vector = word_embeddings[target_word]
    similarities = {}
    for word, vector in word_embeddings.items():
        if word != target_word:
            similarities[word] = cosine_similarity(target_vector, vector)
    # Sort by similarity
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return sorted_similarities[:n]



In [ ]:
get_top_n_similar_words('reward', word_embeddings, n=10)

[('bonus', np.float32(0.54364353)),
 ('ppmpoboxbhambxe', np.float32(0.51361966)),
 ('hotmix', np.float32(0.44465983)),
 ('calloptoutj', np.float32(0.414103)),
 ('todayfrom', np.float32(0.4139428)),
 ('apsaward', np.float32(0.4122826)),
 ('tushin', np.float32(0.40710118)),
 ('freefone', np.float32(0.40527236)),
 ('polyphonic', np.float32(0.40508807)),
 ('dough', np.float32(0.39912173))]

In [ ]:
get_top_n_similar_words('prize', word_embeddings, n=10)

[('magazine', np.float32(0.42165235)),
 ('mssuman', np.float32(0.39212197)),
 ('falconerf', np.float32(0.39149868)),
 ('priest', np.float32(0.36515862)),
 ('thuglyfe', np.float32(0.36444417)),
 ('careabout', np.float32(0.35936666)),
 ('mobstorequizppm', np.float32(0.35568982)),
 ('grahmbell', np.float32(0.352596)),
 ('parish', np.float32(0.35140562)),
 ('nitric', np.float32(0.3357464))]

In [ ]:
cosine_similarity(word_embeddings['phone'], word_embeddings['call']) # did not work as well as expected

np.float32(-0.054397956)

In [ ]:
cosine_similarity(word_embeddings['text'], word_embeddings['message'])

np.float32(0.07887484)

In [ ]:
# Get ntop similar words
def get_top_n_similar_words(target_word, word_embeddings, n=10):
    target_vector = word_embeddings[target_word]
    similarities = {}
    for word, vector in word_embeddings.items():
        if word != target_word:
            similarities[word] = cosine_similarity(target_vector, vector)
    # Sort by similarity
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return sorted_similarities[:n]

get_top_n_similar_words('free', word_embeddings, n=20)

[('limited', np.float32(0.40236196)),
 ('landlineonly', np.float32(0.3769837)),
 ('backdoor', np.float32(0.37456143)),
 ('yard', np.float32(0.35390836)),
 ('textpod', np.float32(0.3136002)),
 ('mood', np.float32(0.31198224)),
 ('broad', np.float32(0.30957416)),
 ('sac', np.float32(0.30698884)),
 ('subscription', np.float32(0.30430934)),
 ('flag', np.float32(0.3036724)),
 ('george', np.float32(0.30110094)),
 ('q', np.float32(0.29500404)),
 ('rival', np.float32(0.29429808)),
 ('optoutdwv', np.float32(0.2913026)),
 ('calloptoutndx', np.float32(0.28420985)),
 ('teacoffee', np.float32(0.28236598)),
 ('esaplanade', np.float32(0.28191766)),
 ('logic', np.float32(0.2723593)),
 ('triple', np.float32(0.27178597)),
 ('nytecalpmsgp', np.float32(0.26528463))]

In [ ]:
get_top_n_similar_words('reward', word_embeddings, n=20)

[('bonus', np.float32(0.54364353)),
 ('ppmpoboxbhambxe', np.float32(0.51361966)),
 ('hotmix', np.float32(0.44465983)),
 ('calloptoutj', np.float32(0.414103)),
 ('todayfrom', np.float32(0.4139428)),
 ('apsaward', np.float32(0.4122826)),
 ('tushin', np.float32(0.40710118)),
 ('freefone', np.float32(0.40527236)),
 ('polyphonic', np.float32(0.40508807)),
 ('dough', np.float32(0.39912173)),
 ('quizclub', np.float32(0.39581454)),
 ('bannfwflyppm', np.float32(0.3954876)),
 ('neo', np.float32(0.3933182)),
 ('triple', np.float32(0.3862275)),
 ('vasa', np.float32(0.38414797)),
 ('subscriptngbpwk', np.float32(0.3800444)),
 ('identify', np.float32(0.3778111)),
 ('oral', np.float32(0.3687912)),
 ('videophone', np.float32(0.36657158)),
 ('smartcall', np.float32(0.35913646))]

In [ ]:
get_top_n_similar_words('phone', word_embeddings, n=20)

[('upgrade', np.float32(0.33815807)),
 ('rental', np.float32(0.31226462)),
 ('mrs', np.float32(0.30610892)),
 ('accessible', np.float32(0.2999165)),
 ('clip', np.float32(0.2913238)),
 ('getting', np.float32(0.28507674)),
 ('stanwood', np.float32(0.2780832)),
 ('worldgnun', np.float32(0.2761979)),
 ('nationwide', np.float32(0.26948124)),
 ('limbo', np.float32(0.2682088)),
 ('digital', np.float32(0.26754203)),
 ('tense', np.float32(0.26622087)),
 ('explosive', np.float32(0.26266536)),
 ('itplspls', np.float32(0.2603224)),
 ('refundedthis', np.float32(0.2581207)),
 ('youcarlo', np.float32(0.25473404)),
 ('anybody', np.float32(0.25124446)),
 ('engine', np.float32(0.24404491)),
 ('cela', np.float32(0.24353784)),
 ('hockey', np.float32(0.24111722))]

In [ ]:
get_top_n_similar_words('crazy', word_embeddings, n=20)

[('multimedia', np.float32(0.4226807)),
 ('alertfrom', np.float32(0.4002517)),
 ('eckankar', np.float32(0.36506042)),
 ('personally', np.float32(0.35783258)),
 ('habbahw', np.float32(0.3513697)),
 ('messenger', np.float32(0.34484679)),
 ('multiply', np.float32(0.3417072)),
 ('recdthirtyeight', np.float32(0.3353375)),
 ('zindgi', np.float32(0.3325146)),
 ('storm', np.float32(0.31303832)),
 ('executive', np.float32(0.313)),
 ('responcewhat', np.float32(0.30844173)),
 ('intrude', np.float32(0.30188596)),
 ('noncomittal', np.float32(0.2933287)),
 ('dayswill', np.float32(0.28464863)),
 ('punch', np.float32(0.28203005)),
 ('logic', np.float32(0.28048545)),
 ('specify', np.float32(0.27729613)),
 ('border', np.float32(0.27591735)),
 ('anybody', np.float32(0.2737102))]